201911181 이승민

### 0. 라이브러리 임포트 & spark 세팅

In [1]:
import pyspark
myConf = pyspark.SparkConf()
spark = pyspark.sql.SparkSession\
.builder\
.master('local')\
.appName('myApp')\
.config(conf=myConf)\
.getOrCreate()

21/12/18 19:58:18 WARN Utils: Your hostname, Kritiasui-MacBookAir.local resolves to a loopback address: 127.0.0.1; using 172.30.1.23 instead (on interface en0)
21/12/18 19:58:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/12/18 19:58:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/18 19:58:20 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/12/18 19:58:20 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
21/12/18 19:58:20 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [38]:
import os
from pyspark.sql.types import *
from pyspark.sql import functions as F

# 문제 1

## 문제 1-1의 답

In [43]:
path = os.path.join('data', 'myData.csv')
myDf = spark.read.option('charset', 'euc-kr')\
    .option('header', 'true')\
    .csv(path)
myDf.printSchema()

root
 |-- 사용월: string (nullable = true)
 |-- 호선명: string (nullable = true)
 |-- 지하철역: string (nullable = true)
 |-- 유임승차인원: string (nullable = true)
 |-- 무임승차인원: string (nullable = true)
 |-- 유임하차인원: string (nullable = true)
 |-- 무임하차인원: string (nullable = true)
 |-- 작업일자: string (nullable = true)



## 문제 1-2의 답

In [44]:
myDf.count()

55171

## 문제 1-3의 답

In [45]:
from pyspark.sql.window import Window
myDf = myDf.withColumn('i유임승차인원', F.col('유임승차인원').cast(IntegerType()))
myDf = myDf.drop('유임승차인원').withColumnRenamed('i유임승차인원', '유임승차인원')

In [48]:
win = Window.partitionBy('호선명').orderBy(F.col('유임승차인원').desc())
myDf = myDf.withColumn('rank', F.rank().over(win))
myDf.show(30)

+------+------+--------+------------+------------+------------+--------+------------+----+
|사용월|호선명|지하철역|무임승차인원|유임하차인원|무임하차인원|작업일자|유임승차인원|rank|
+------+------+--------+------------+------------+------------+--------+------------+----+
|201605|일산선|    화정|      139163|      535460|      142410|20160608|      515925|   1|
|201512|일산선|    화정|      136734|      529106|      139931|20160108|      508044|   2|
|201603|일산선|    화정|      140596|      526030|      144088|20160408|      502144|   3|
|201703|일산선|    화정|      141327|      529944|      145502|20170403|      499911|   4|
|201510|일산선|    화정|      520595|      140545|      144148|20151108|      499388|   5|
|201905|일산선|    화정|      149790|      520966|      152915|20190603|      499218|   6|
|201612|일산선|    화정|      132807|      521734|      136086|20170108|      497514|   7|
|201805|일산선|    화정|      139239|      515748|      142997|20180603|      494994|   8|
|201610|일산선|    화정|      137462|      513849|      140303|20161108|      4921

## 문제 1-4의 답

In [53]:
myDf[myDf['rank'] == 1].show(30)

+------+--------------+--------------------+------------+------------+------------+--------+------------+----+
|사용월|        호선명|            지하철역|무임승차인원|유임하차인원|무임하차인원|작업일자|유임승차인원|rank|
+------+--------------+--------------------+------------+------------+------------+--------+------------+----+
|201605|        일산선|                화정|      139163|      535460|      142410|20160608|      515925|   1|
|201905|        장항선|                아산|       12901|       93737|       11808|20190603|      109243|   1|
|201512|        경부선|              영등포|      280623|     1545631|      280969|20160108|     1439049|   1|
|201905|    우이신설선|        북한산보국문|       45041|      141742|       45814|20190603|      161867|   1|
|201512|        분당선|                야탑|      157846|      809197|      156034|20160108|      769442|   1|
|201512|         7호선|      가산디지털단지|       82588|     1243464|       78424|20160108|     1245731|   1|
|201905|        수인선|              인하대|       28246|      189885|       26442|201

## 문제 1-5의 답

In [68]:
import sys
allWindow = Window.rowsBetween(-sys.maxsize, sys.maxsize)
byLine = Window.partitionBy('유임승차인원')
_myDf = myDf.withColumn('meanByLine', F.avg(myDf['유임승차인원']).over(byLine))
_myDf = _myDf.withColumn('sdByLine', F.stddev(myDf['유임승차인원']).over(byLine))
_myDf = _myDf.withColumn('zscore', (F.col('유임승차인원') - F.col('meanByLine'))/ F.col('sdByLine'))
_myDf.select('호선명', '유임승차인원', 'meanByLine', 'sdByLine', 'zscore').show()

+----------+------------+----------+--------+------+
|    호선명|유임승차인원|meanByLine|sdByLine|zscore|
+----------+------------+----------+--------+------+
|    경원선|        1580|    1580.0|    null|  null|
|    중앙선|        4519|    4519.0|    null|  null|
|    경춘선|        4818|    4818.0|    null|  null|
|    경강선|        5518|    5518.0|    null|  null|
|    중앙선|        6336|    6336.0|    null|  null|
|    경춘선|        7554|    7554.0|    null|  null|
|    중앙선|       14570|   14570.0|    null|  null|
|     4호선|       18498|   18498.0|    null|  null|
|    중앙선|       20924|   20924.0|    null|  null|
|    경춘선|       22097|   22097.0|    null|  null|
|    중앙선|       23015|   23015.0|    null|  null|
|    장항선|       23364|   23364.0|    null|  null|
|    경춘선|       24347|   24347.0|    null|  null|
|    경춘선|       28124|   28124.0|    null|  null|
|우이신설선|       28146|   28146.0|    null|  null|
|    일산선|       31261|   31261.0|    null|  null|
|우이신설선|       32460|   32460.0|    null|  null|
|  

# 2번

In [69]:
from sklearn.datasets import make_regression

X, y, coef = make_regression(n_samples = 200, n_features = 4, n_informative = 3, n_targets = 1, noise = 0.0, coef = True)

In [72]:
X.shape

(200, 4)

In [74]:
import numpy as np

def gradientDescent(x, y, theta, alpha, numIterations):
    for i in range(numIterations):
        h=np.dot(x, theta)
        error=h-y
        cost=np.sum((h-y)**2) / len(x)
        gradient=np.dot(x.T,error)*2/len(x)
        theta-=alpha*gradient
        if (i%100==0):
            print ("iter:{0:5d}\ttheta {1} Cost {2:.5f}".format(i, theta, cost))
    return theta

theta = np.ones(X.shape[1])
alpha = 0.01
iter = 999

## 2-1의 정답

In [79]:
result = gradientDescent(X, y, theta, alpha, iter)
print('theta: {}\ncoef : {}'.format(result, coef))

iter:    0	theta [7.42670134e+01 4.30798370e-14 5.47295915e+01 6.85430070e+01] Cost 0.00000
iter:  100	theta [7.42670134e+01 4.30795534e-14 5.47295915e+01 6.85430070e+01] Cost 0.00000
iter:  200	theta [7.42670134e+01 4.30792699e-14 5.47295915e+01 6.85430070e+01] Cost 0.00000
iter:  300	theta [7.42670134e+01 4.30789864e-14 5.47295915e+01 6.85430070e+01] Cost 0.00000
iter:  400	theta [7.42670134e+01 4.30787028e-14 5.47295915e+01 6.85430070e+01] Cost 0.00000
iter:  500	theta [7.42670134e+01 4.30797671e-14 5.47295915e+01 6.85430070e+01] Cost 0.00000
iter:  600	theta [7.42670134e+01 4.30794835e-14 5.47295915e+01 6.85430070e+01] Cost 0.00000
iter:  700	theta [7.42670134e+01 4.30792000e-14 5.47295915e+01 6.85430070e+01] Cost 0.00000
iter:  800	theta [7.42670134e+01 4.30789165e-14 5.47295915e+01 6.85430070e+01] Cost 0.00000
iter:  900	theta [7.42670134e+01 4.30786329e-14 5.47295915e+01 6.85430070e+01] Cost 0.00000
theta: [7.42670134e+01 4.30792176e-14 5.47295915e+01 6.85430070e+01]
coef : [74.

비교: e표기법과 precision을 고려했을 때, 1.0e-06까지도 완벽히 일치한다.

## 2-2의 정답

In [112]:
from pyspark.mllib.regression import LabeledPoint
#spark.sparkContext.parallelize([X, y][0]).collect()
p = spark.sparkContext.parallelize(np.c_[X, y].tolist()).map(lambda x: [x[4], x[0], x[1], x[2], x[3]]).collect()
regDf = spark.createDataFrame(p, ['label', '0', '1', '2', '3'])
regDf.printSchema()

root
 |-- label: double (nullable = true)
 |-- 0: double (nullable = true)
 |-- 1: double (nullable = true)
 |-- 2: double (nullable = true)
 |-- 3: double (nullable = true)



## 2-3의 정답

In [113]:
train, test = regDf.randomSplit([0.6, 0.4])
train.count(), test.count()

(123, 77)

## 2-4의 정답

In [117]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
va = VectorAssembler(inputCols = ['0', '1', '2', '3'], outputCol = 'features')
lr = LinearRegression(featuresCol= 'features', labelCol='label', regParam = 0.3)
pipe = Pipeline(stages = [va, lr])

In [118]:
model = pipe.fit(train)
modelTrainDf = model.transform(train)

In [120]:
print(f'계수: {model.stages[-1].coefficients}\n절편: {model.stages[-1].intercept}')

계수: [74.09099684760473,-0.017827685057547706,54.57826493514655,68.361389395412]
절편: 0.022760266248761388


## 2-5의 정답

In [123]:
predictDf = model.transform(test)
predictDf.select('label', 'prediction').show(10)

+-------------------+-------------------+
|              label|         prediction|
+-------------------+-------------------+
|-271.82399152124515|-271.12746023757296|
|-243.29785695943897|-242.67083779679365|
| -203.2411663154771| -202.6832498263636|
|-182.80875618802804|-182.32493315948338|
| -160.0358893154457|-159.59440156866688|
| -151.0358225348507|-150.59257288286264|
|-147.63028516080954|-147.23685931325844|
|-138.72700790926731|-138.37877711304202|
|-120.44878726263362|-120.11430733035846|
|-107.73438186080229|-107.39699589924034|
+-------------------+-------------------+
only showing top 10 rows



# 3번

## 3-1의 답

In [131]:
myD = []
with open(os.path.join('data', 'movie.txt')) as f:
    for line in f.readlines():
        myD.append((line[0], line[1:].strip()))

moDf = spark.createDataFrame(myD, ['긍부정', '리뷰'])
moDf.printSchema()

root
 |-- 긍부정: string (nullable = true)
 |-- 리뷰: string (nullable = true)



## 3-2의 답

In [135]:
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover
tok = Tokenizer(inputCol='리뷰', outputCol= 'words')
tokDf = tok.transform(moDf)
stop = StopWordsRemover(inputCol = 'words', outputCol= 'nostops')
stopList = ['!', '.', 'ㅋㅋㅋ', 'ㅠ', 'ㄹㅇ', '그', '수', '본', '등', '때', '걸', '다', '게', '날', '짱', '거', '중', '더', '넘', '두', '달', '올', '뻔', '단', '개', '듯']
stop.setStopWords(stopList)
stopDf = stop.transform(tokDf)
stopDf.show()

+------+----------------------------------+--------------------------------+--------------------------------+
|긍부정|                              리뷰|                           words|                         nostops|
+------+----------------------------------+--------------------------------+--------------------------------+
|     0|고구마 먹은 영화 유치해서 못보겠다|  [고구마, 먹은, 영화, 유치해...|  [고구마, 먹은, 영화, 유치해...|
|     0|  노잼이네요.. 왜 배우들이 출연...|  [노잼이네요.., 왜, 배우들이...|  [노잼이네요.., 왜, 배우들이...|
|     0|   제발 뻘짓 좀 그만하셨으면......|  [제발, 뻘짓, 좀, 그만하셨으...|  [제발, 뻘짓, 좀, 그만하셨으...|
|     0|         스토리가 매우 매우 아쉽다|  [스토리가, 매우, 매우, 아쉽다]|  [스토리가, 매우, 매우, 아쉽다]|
|     0|             영화 약간 이해가 안됨|      [영화, 약간, 이해가, 안됨]|      [영화, 약간, 이해가, 안됨]|
|     0|좋아하는 배우들인데 최근 영화라...| [좋아하는, 배우들인데, 최근,...| [좋아하는, 배우들인데, 최근,...|
|     0| 그지같네 도대체 이 따위 영화를...|  [그지같네, 도대체, 이, 따위...|  [그지같네, 도대체, 이, 따위...|
|     0|  스크린보다 시계를 더 많이 봄....|  [스크린보다, 시계를, 더, 많...|  [스크린보다, 시계를, 많이, ...|
|     0| 이때까지 본 영화중에 제일 최악...|  [이때까지, 본, 영화중에, 제...| 

## 3-3의 답

In [139]:
from pyspark.ml.feature import HashingTF, IDF

hashTF = HashingTF(inputCol = 'nostops', outputCol = 'hash')
hashDf = hashTF.transform(stopDf)

In [141]:
idf = IDF(inputCol = 'hash', outputCol = 'idf')

idfModel = idf.fit(hashDf)
idfDf = idfModel.transform(hashDf)
idfDf.select('nostops', 'hash').show(5, truncate = False)

+-----------------------------------------------------------+---------------------------------------------------------------------------------------+
|nostops                                                    |hash                                                                                   |
+-----------------------------------------------------------+---------------------------------------------------------------------------------------+
|[고구마, 먹은, 영화, 유치해서, 못보겠다]                   |(262144,[114067,145746,202865,232307,243237],[1.0,1.0,1.0,1.0,1.0])                    |
|[노잼이네요.., 왜, 배우들이, 출연했는지, 이해가, 잘, 안됨] |(262144,[14123,77545,102360,123815,178388,221856,252373],[1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
|[제발, 뻘짓, 좀, 그만하셨으면...., 이건, 여러모로, 낭비다.]|(262144,[30761,45298,104569,160982,165201,196501,249777],[1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
|[스토리가, 매우, 매우, 아쉽다]                             |(262144,[8121,118336,181130],[1.0,2.0,1.0])                                            |
|[영화, 약간, 이해가, 안됨

## 3-4의 답

In [152]:
from pyspark.ml.classification import NaiveBayes
_idfDf = idfDf.withColumn('label', F.col('긍부정').cast(IntegerType()))
nb = NaiveBayes(featuresCol = 'hash', labelCol = 'label', modelType = 'multinomial', predictionCol = 'prediction')
model = nb.fit(_idfDf)


## 3-5의 답

In [154]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

idfDf = _idfDf
trainDf, testDf = idfDf.randomSplit([0.5, 0.5])
model = nb.fit(trainDf)
predictions = model.transform(testDf)

evaluator = BinaryClassificationEvaluator(rawPredictionCol= 'prediction', labelCol = 'label')
evaluator.evaluate(predictions)

21/12/18 22:47:51 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/12/18 22:47:51 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB
21/12/18 22:47:55 WARN DAGScheduler: Broadcasting large task binary with size 8.1 MiB


0.65

## 3-6의 답

In [160]:
myTest = [(-1, '고구가 먹은 느낌 킬링타음으로도 아쉽다'),
          (-1, '내돈 아깝 핵노잼 영화가 이렇게 지루해서 못보겠다'),
          (-1, '보다가 채널 돌림 재미없다'),
          (-1, '최고의 영화'),
          (-1, '여운이 긴 명작'),
          (-1, '띵작입니다 몰입하고 숨도 못쉬고 봤어요 꼭 보세요')]
realTestDf = spark.createDataFrame((myTest), ['긍부정', '리뷰'])
realTestDf = tok.transform(realTestDf)
realTestDf = stop.transform(realTestDf)

In [ ]:
realTestDf = hashTF.transform(realTestDf)
realTestDf = idfModel.transform(realTestDf)

In [163]:
predictions = model.transform(realTestDf)
predictions.show()

+------+----------------------------------+-------------------------------+-------------------------------+--------------------+--------------------+--------------------+--------------------+----------+
|긍부정|                              리뷰|                          words|                        nostops|                hash|                 idf|       rawPrediction|         probability|prediction|
+------+----------------------------------+-------------------------------+-------------------------------+--------------------+--------------------+--------------------+--------------------+----------+
|    -1|고구가 먹은 느낌 킬링타음으로도...| [고구가, 먹은, 느낌, 킬링타...| [고구가, 먹은, 느낌, 킬링타...|(262144,[8121,745...|(262144,[8121,745...|[-60.776859164829...|[0.91828727620625...|       0.0|
|    -1| 내돈 아깝 핵노잼 영화가 이렇게...| [내돈, 아깝, 핵노잼, 영화가...| [내돈, 아깝, 핵노잼, 영화가...|(262144,[35595,47...|(262144,[35595,47...|[-83.651936454055...|[0.98899473334772...|       0.0|
|    -1|         보다가 채널 돌림 재미없다| [보다가, 채널, 돌림, 재미없다]| [보다

21/12/18 22:57:00 WARN DAGScheduler: Broadcasting large task binary with size 8.1 MiB
